In [1]:
"""Manejo de Informacion"""

import pandas as pd
from pandas import json_normalize
import requests
import json

"""Tiempo"""

from datetime import datetime
from datetime import timezone

"""Textos"""

import re 
from unidecode import unidecode
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from itertools import chain
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

"""Visualizaciones"""

import matplotlib.pyplot as plt
import seaborn as sns

"""ML"""

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB



pd.options.mode.chained_assignment = None 

# Limpiar datos

In [2]:
def CSV_transform(df):
    """
    Función que transforma de formato CSV a diccionario
    """
    
    df = df[~(df["lyrics"] == "error")] # No tomar en cuenta canciones que no tienen letra
    
    df["lyrics"] = df.lyrics.str.replace("\r"," ").str.replace("\n"," ").str.replace("[}{&:;,.¡!¿?\(\)\-\"\"0-9]","").str.replace("[","").str.replace("]","").str.lower() # Quitar espacios, interlineados, reemplazar algunos signos/numeros y pasar a minúsculas.
    
    df["lyrics"] = df.lyrics.apply(lambda x: unidecode(x)) # Quitar unicodes de la forma \uxxxx
    
    df["lyrics"] = df.lyrics.apply(lambda x: " ".join(x.split())) # Strippear el texto (quitar espacios innecesarios)
     
    return df

# Obtener Datos

Se tienen 3 datasets disponibles por si no se quiere/puede obtener la información personal.

In [3]:
df1 = CSV_transform(pd.read_csv("top_david_spotify.csv",usecols = ["name","lyrics"]).dropna().reset_index(drop = True))
df2 = CSV_transform(pd.read_csv("top_javier_spotify.csv",usecols = ["name","lyrics"]).dropna().reset_index(drop = True))
df3 = CSV_transform(pd.read_csv("top_jesus_spotify.csv",usecols = ["name","lyrics"]).dropna().reset_index(drop = True))

In [4]:
def Obtener_datos(numero):
    if numero == 1:
        return df1
    if numero == 2:
        return df2
    if numero == 3:
        return df3

# Naive Bayes Classifier para Identificar idioma

En este tópico utilizo:

1. __Tema 13: clasificador de Lengua (Naïve Bayes)__
2. __Tema 4: Matriz de Incidencia (frecuencias)__

Entrenar algorítmo de clasificación para clasificar entre 17 lenguas. Se utilizará el algorítmo visto en clase, pero implementado por Sklearn.

[El conjunto de datos etiquetado](https://www.kaggle.com/datasets/basilb2s/language-detection) fue extraido de Kaggle para facilitar el etiquetado. 

1) English
2) Malayalam
3) Hindi
4) Tamil
5) Kannada
6) French
7) Spanish
8) Portuguese
9) Italian
10) Russian
11) Sweedish
12) Dutch
13) Arabic
14) Turkish
15) German
16) Danish
17) Greek

__NOTA__: Dado que la longitud de las canciones no es tan extensa, no se aplicará ningún tipo de stemming. Tampoco considero necesario aplicar la técnica de los bigramas. 



In [5]:
# Cargar Dataset
lenguajes = pd.read_csv("Language Detection.csv")

# Realizar Matriz de Incidencias

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(lenguajes.Text.str.replace("[{}:;,.¡!¿?\(\)\"\"0-9]","").to_list())

# Crear modelo 

NB = MultinomialNB() # Dejar prior como uniforme
NB.fit(X, lenguajes.Language.values)

MultinomialNB()

In [6]:
def Identificar_Idioma(df):
    X_test  = vectorizer.transform(df.lyrics)
    
    df["Idioma"] = NB.predict(X_test)
    
    return "Exitoso Identificador de Idioma"


# Completado de palabras en español en caso de contracción

En este tópico utilizo:

1. __Tema 6: Levenshtein Metric__

En español, como bien sabemos, no existen formalmente; sin embargo, pragmáticamente se ha adquirido la costumbre de "recortar" algunas palabras y la forma de representar este fenómeno es por medio de un __'__. Comunmente estas contracciones se efectuan __en preposiciones__, esto es, stopwords. En cuanto a RI no son relevantes, pero para la interpretación literaria del texto, sí.

Se pretende identificar estas palabras y completarlas por medio de una lista de palabras comunmente contraidas (informalmente) en el español. 

In [25]:
def Contracciones_español(df,porcentaje = 60, lista_extra = []): 
    assert "Spanish" in df["Idioma"].unique(), 'No escuchas música en español'
    
    lista = lista_extra + [palabra for palabra in stopwords.words("Spanish") if len(palabra) >=3]
    
    español = df[df["Idioma"] == "Spanish"]["lyrics"]
    
    for indice in español.index:
            palabras_cancion = español[indice].split()
            
            for index in range(len(palabras_cancion)):
                if "'" in palabras_cancion[index]:
                    try:
                        palabras_cancion[index] =  process.extractOne(palabras_cancion[index], lista,score_cutoff = porcentaje)[0]
                    except:
                        pass
                else:
                    pass
            
            cancion_corregida = " ".join(palabras_cancion)
            
            df.iloc[indice,1] = cancion_corregida
    
    return "Exitosa corrección de palabras en la lista"

# Contracciones (')

En este tópico utilizo:

1. __Tema 3: Regex (Expresiones regulares)__

Dado que en algunas lenguas romances el uso de __'__ resulta determinante para el contexto de la oración, no puede ser fácilmente eliminado del corpus. En adición, las contracciones en el idioma inglés también existen y son muy comunes.  En general, en caso de que la contracción sea entre una preposición y una palabra relevante, es más probable que la palabra sea de mediana longitud. La función está primordialmente orientada a lenguas romance (incluyendo inglés) que las utilicen.

Se pretende identificar contracciones útiles por medio de la identificación de la longitud de la segunda palabra.

# Tokenizar Canciones

In [ ]:
def Tokenizar(df):
    df["tokens"] = df["lyrics"].apply(lambda x: set(nltk.word_tokenize(x))) # Tokenizar las canciones
    
    return "Exitoso Tokenizado"

# Eliminar palabras de longitud mayor a...

In [ ]:
def Eliminar_mayor_len(df,limite):
    limpias = []
    for i in range(df.shape[0]):
        limpias.append({palabra_menos for palabra_menos in df.tokens[i] if len(palabra_menos) < limite})
    
    df["tokens"] = limpias
    
    return f"Exitosa eliminación de palabras con longitud mayor a {limite}"
                        

# Eliminar palabras de longitud menor a...

In [ ]:
def Eliminar_menor_len(df,limite):
    limpias = []
    for i in range(df.shape[0]):
        limpias.append({palabra_menos for palabra_menos in df.tokens[i] if len(palabra_menos) > limite})
    
    df["tokens"] = limpias
    
    return f"Exitosa eliminación de palabras con longitud menor a {limite}"

# Stopwords

In [ ]:
def Stopwords(df):
    # Lista de idiomas
    idiomas = df.Idioma.unique()
    
    
    for idioma in idiomas:
        try:
            stopwords_ = stopwords.words(idioma) # Stopwords
        except:
            print("No hay stopwords para", idioma)
            
        canciones_idioma = df[df["Idioma"] == idioma]["tokens"]
        
        for indice in canciones_idioma.index:
            canciones_idioma[indice] = [palabra for palabra in canciones_idioma[indice] if palabra not in stopwords_]
            
        test.iloc[canciones_idioma.index,2] = canciones_idioma
        
    return "Exitosa eliminación de stopwords por idiomas identificados"
            

---

---

In [12]:
test = Obtener_datos(2)

In [13]:
Identificar_Idioma(test)

'Exitoso Identificador de Idioma'

In [26]:
Contracciones_español(test)

'Exitosa corrección de palabras en la lista'

In [ ]:
Contracciones(test)

In [ ]:
Tokenizar(test)

In [ ]:
Eliminar_mayor_len(test,10)

In [ ]:
Eliminar_menor_len(test,3)

In [ ]:
Stopwords(test)

In [ ]:
for i in [palabra for palabra in FreqDist(list(chain(*test["tokens"].to_list()))).keys() if "'" in palabra]: #[test["Idioma"] == "English"]:
    print(i)